#### Generate a RxSqlServerData data source object

https://msdn.microsoft.com/en-us/microsoft-r/scaler/rxsqlserverdata

In [1]:
library(RevoScaleR)

In [2]:
#connStr
instance_name <- "TRB_MICROSOFT"
database_name <- "WideWorldImportersDW"
connStr <- paste("Driver={SQL Server Native Client 11.0};Server=",
                 instance_name, ";Database=",database_name,";Trusted_Connection=yes;",sep="");

In [3]:
## Open a connection with SQL Server to be able to write queries with the rxExecuteSQLDDL function.

outOdbcDS <- RxOdbcData(table = "NewData", connectionString = connStr, useFastRead=TRUE)
rxOpen(outOdbcDS, "w")

[1] TRUE

In [4]:
# Fails since the column names have embedded blanks
SqlQ0 <- RxSqlServerData(sqlQuery = "SELECT WWI Customer ID,
                                            Customer,
                                            Category,
                                            Buying Group,
                                            Postal Code
                                     FROM Dimension.Customer",
                        connectionString = connStr)
rxGetInfo(SqlQ0, getVarInfo = TRUE)

[Microsoft][SQL Server Native Client 11.0][SQL Server]Incorrect syntax near 'ID'.
[Microsoft][SQL Server Native Client 11.0][SQL Server]Statement(s) could not be prepared.
[Microsoft][SQL Server Native Client 11.0][SQL Server]Incorrect syntax near 'ID'.
[Microsoft][SQL Server Native Client 11.0][SQL Server]Statement(s) could not be prepared.


ERROR: Error in doTryCatch(return(expr), name, parentenv, handler): [Microsoft][SQL Server Native Client 11.0][SQL Server]Incorrect syntax near 'ID'.
[Microsoft][SQL Server Native Client 11.0][SQL Server]Statement(s) could not be prepared.




In [5]:
# Undeseriable behavior since single quoted column names fail to fetch the SQL/Server column names
SqlQ1 <- RxSqlServerData(sqlQuery = "SELECT 'WWI Customer ID',
                                            'Customer',
                                            'Category',
                                            'Buying Group',
                                            'Postal Code'
                                     FROM Dimension.Customer",
                        connectionString = connStr)
rxGetInfo(SqlQ1, getVarInfo = TRUE)

Connection string: Driver={SQL Server Native Client 11.0};Server=TRB_MICROSOFT;Database=WideWorldImportersDW;Trusted_Connection=yes; 
Data Source: SQLSERVER 
Number of variables: 5 
Variable information: 
Var 1: column1, Type: character
Var 2: column2, Type: character
Var 3: column3, Type: character
Var 4: column4, Type: character
Var 5: column5, Type: character

In [6]:
#desired behavior since double quoted column names fetches the SQL Server column names
SqlQ2 <- RxSqlServerData(sqlQuery = 'SELECT "WWI Customer ID",
                                            "Customer",
                                            "Category",
                                            "Buying Group",
                                            "Postal Code"
                                     FROM Dimension.Customer',
                        connectionString = connStr)
rxGetInfo(SqlQ2, getVarInfo = TRUE)

Connection string: Driver={SQL Server Native Client 11.0};Server=TRB_MICROSOFT;Database=WideWorldImportersDW;Trusted_Connection=yes; 
Data Source: SQLSERVER 
Number of variables: 5 
Variable information: 
Var 1: WWI Customer ID, Type: integer
Var 2: Customer, Type: character
Var 3: Category, Type: character
Var 4: Buying Group, Type: character
Var 5: Postal Code, Type: character

In [ ]:
rxGetInfo(SqlQ2, getVarInfo = TRUE)

In [ ]:
CustDf1 <- rxImport(SqlQ1)

In [ ]:
is.data.frame(CustDf1)